In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

#md_file = '/home/jvdzwaan/data/adh-corpora/fiqh_corpus/Meta/Metadata_Fiqh.csv'
md_file = '/home/jvdzwaan/data/adh-corpora/dawa/DawaMetadata.csv'
#md_file = '/media/sf_VBox_Shared/Arabic/fiqh_corpus/Meta/Metadata_Fiqh.csv'

In [ ]:
md = pd.read_csv(md_file, sep=';', encoding='utf-8', index_col=0)
md.head()

In [ ]:
# replace \ as a separator between authors (blacklab/corpus-frontend don't like it)

md['Author'] = md['Author'].fillna('UNKNOWN')
md['Editor'] = md['Editor'].fillna('UNKNOWN')
md['Author'] = md.apply(lambda row: row['Author'].replace('/', '+'), axis=1)
md['Publisher'] = md['Publisher'].fillna('UNKNOWN')
md['Weblink'] = md['Weblink'].fillna('UNKNOWN')
md['Country'] = md['Country'].fillna('UNKNOWN')
md['Place'] = md['Place'].fillna('UNKNOWN')

In [ ]:
md.head()

In [ ]:
md['Collection'] = md['Collection'].fillna('NONE')
md['Extra'] = md['Extra'].fillna('NONE')
md.head()

In [ ]:
md['Chapter'] = md['Chapter'].fillna(-1)
md['Chapter'] = pd.to_numeric(md['Chapter'], downcast='integer')

md['Section'] = md['Section'].fillna(-1)
md['Section'] = pd.to_numeric(md['Section'], downcast='integer')

In [ ]:
md.head()

In [ ]:
md['Year'] = md['Year'].fillna(-1)
md['Year'] = pd.to_numeric(md['Year'], downcast='integer')

In [ ]:
num = 105

md.at[num, 'Birthdate'] = -1
md.at[num, 'Deathdate'] = -1
md.loc[num]

In [ ]:
md.at[2611, 'Birthdate'] = -1
md.at[2611, 'Deathdate'] = -1
md.loc[2611]

In [ ]:
def update_birthdate(row):
    if row['Birthdate'] == 'UNKNOWN':
        return -1
    return row['Birthdate']

md['Birthdate'] = md.apply(lambda row: update_birthdate(row), axis=1)

In [ ]:
md['Birthdate'] = md['Birthdate'].fillna(-1)
md['Birthdate'] = pd.to_numeric(md['Birthdate'], downcast='integer')
md.head()

In [ ]:
md.columns
md = md.drop('Daethdate', axis=1)

In [ ]:
md.at[2700, 'Deathdate'] = -1

In [ ]:
for idx in md.query('Deathdate == "  "').index:
    md.at[idx, 'Deathdate'] = -1

In [ ]:
def update_deathdate(row):
    if row['Deathdate'] == 'UNKNOWN':
        return -1
    return row['Deathdate']
md['Deathdate'] = md.apply(lambda row: update_deathdate(row), axis=1)

In [ ]:
md['Deathdate'] = md['Deathdate'].fillna(-1)
md['Deathdate'] = pd.to_numeric(md['Deathdate'], downcast='integer')

In [ ]:
# put book title in collection field for works other than 262 and 278
def get_collection(row):
    if row['Collection'] == 'NONE':
        print(row['Title'])
        return row['Title']
    return row['Collection']
    
md['Collection'] = md.apply(lambda row: get_collection(row), axis=1)

In [ ]:
len(md['Collection'].unique())

In [ ]:
# Do not update the notebook with this! It will add X-es to long author names
def too_long(row, max_len=94):
    if len(row['Author']) >= max_len:
        #print('Too long', len(row['Author']))
        print(row['BookURI'])
        return 'X ' + row['Author'][:max_len-2]
    return row['Author']

md.apply(lambda row: too_long(row), axis=1)

In [ ]:
def check(row):
    if row['BookURI'].startswith('262_'):
        return True
    return False

md['auth_len'] = md.apply(lambda row: len(row['Author']), axis=1)
md['check'] = md.apply(lambda row: check(row), axis=1)
md.head()

In [ ]:
md.query('check==False')['auth_len'].max()

In [ ]:
def author_repr(row):
    if row['BookURI'].startswith('262_'):
        #print(repr(row['Author']))
        return True
    return False
md.apply(lambda row: author_repr(row), axis=1)

In [ ]:
from collections import Counter

from adhtools.split_text_openiti_markers import smart_strip

c = Counter(list(md[md.apply(lambda row: author_repr(row), axis=1)]['Author']))

for t, f in c.most_common():
    print(repr(t))
    print(t)
    print(repr(smart_strip(t)))
    print(smart_strip(t))
    print(f)
    print()

In [ ]:
len(md['Author'].unique())

In [ ]:
md.sample(10)

In [ ]:
md.to_csv(md_file)